In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import time
from pprint import pprint
import re
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

events_by_season_dir = '../input/baseball-events-from-retrosheetorg/events_by_season/'
metadata_dir = '../input/baseball-events-from-retrosheetorg/'


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

thisyear = 1916
#first_eventID = 0    # 1915
first_eventID = 104242 + 1   # 1916
#first_eventID = 172315 + 1   # 1917

if (thisyear == 1915):
    league_list = ['F', 'A', 'N']
else:
    league_list = ['A', 'N']

new_columns = ['vis_score', 'home_score', 'outs', 'runner_1b', 'runner_2b', 'runner_3b']

final_column_order = ['gameID',
 'event_in_game',
 'inning',
 'visitor_or_home',
  'vis_score',
 'home_score',
 'outs',
 'runner_1b',
 'runner_2b',
 'runner_3b',
 'batterID',
 'batter_hand',
 'pitch_count',
 'pitch_sequence',
 'play_orig',
 'pitcherID',
 'pitcher_hand',
 'field_C_playerID',
 'field_1B_playerID',
 'field_2B_playerID',
 'field_3B_playerID',
 'field_SS_playerID',
 'field_LF_playerID',
 'field_CF_playerID',
 'field_RF_playerID',
 'bat_DH_playerID',
 'pinch_runner_playerID',
 'pinch_runner_replaced_playerID',
 'comments',
 'badj',
 'padj',
 'theyear',
 'visteam',
 'hometeam',
 'baserunning',
 'play_description',
 'theplay',
 'baserunning_cleaned']

early_end_game_list = ['PHI191507210', 'CHN191607180', 'NY1191609080', 'NY1191609182', 'PHA191905050']
# early_end_game_list += ['CHA192104210']
# early_end_game_list += ['CIN202106300', 'PIT202106100', 'SLN202110030']

special_case_events = {}
special_case_events.update({'PHA191509242': [17], 'CHN191509242': [37], 'PHI191509092': [40], 'PIT191506080': [47]})
special_case_events.update({'PHA191605170': [24]})

#, 'PIT191606242': [58]})#, 'SLN191606290': [5]})
# # special_case_events.update({'BOS191707261': [49], 'PHI191710022': [25]})
# # special_case_events.update({'CHA191806200': [67], 'CHA191907100': [52], 'DET191906021': [28], 'NY1191908261': [31]})
# # special_case_events.update({'NY1192008151': [66], 'PHI192005060': [5], 'PHI192009100': [53]})

# # special_case_events.update({'CHN192108032': [63], 'CIN192104180': [31], 'CIN192104240': [55], 'CIN192106110': [26], 'NY1192105040': [39], 'NY1192105240': [22], 'NY1192105302': [85], 'NY1192106210': [57]})
# # special_case_events.update({'ARI202107310': [16], 'ARI202108300': [5], 'ATL202106201': [32], 'CHN202107080': [13], 'CIN202104160': [32]})

def ordinaltg(n):
    return str(n) + {1: 'st', 2: 'nd', 3: 'rd'}.get(4 if 10 <= n % 100 < 20 else n % 10, "th")
g = 0

print('ok')
#special_case_events

ok


In [3]:
s = time.time()
print('Adding game context data for games in {0:.0f}...'.format(thisyear))
print('\n')
print('reading rosters...')
rosters_df = pandas.read_csv(metadata_dir+'rosters.csv', low_memory=False, index_col='rosterID')

print('reading games...')
games_df = pandas.read_csv(metadata_dir+'games.csv', low_memory=False, index_col='gameID', encoding='utf-8', converters={'visitor_linescore': lambda x: str(x), 'home_linescore': lambda x: str(x)})

print('\tfixing linescores...')
# games_df.loc[games_df['visitor_linescore'] == '"nan"', 'visitor_linescore'] = np.nan
# games_df.loc[games_df['home_linescore'] == '"nan"', 'home_linescore'] = np.nan
# games_df.loc[:, 'visitor_linescore'] = games_df['visitor_linescore'].apply(lambda x: str(str(x)[1:-1]))
# games_df.loc[:, 'home_linescore'] = games_df['home_linescore'].apply(lambda x: str(str(x)[1:-1]))
games_df = games_df.assign(has_linescores = True)
games_df.loc[(games_df['visitor_linescore'].isnull()) | (games_df['home_linescore'].isnull()), 'has_linescores'] = False

print('\tchecking for DH...')
has_dh = True
if (thisyear < 1973):
    has_dh = False

print('\tgetting fielder columns...')
fielder_columns = ['field_C_playerID', 'field_1B_playerID', 'field_2B_playerID', 'field_3B_playerID', 'field_SS_playerID']
fielder_columns += ['field_LF_playerID', 'field_CF_playerID', 'field_RF_playerID'] 
if (has_dh):
    fielder_columns += ['bat_DH_playerID']


print('selecting games and roster rows for {0:.0f}...'.format(thisyear))
games_df.loc[:, 'date'] = pandas.to_datetime(games_df['date'], format='%Y%m%d', errors='coerce')
games_df = games_df[games_df['date'].apply(lambda x: x.year == thisyear)]
rosters_df = rosters_df[rosters_df['theyear'] == thisyear]

events_df = pandas.DataFrame()
eventfile_list = []
for thisleague in league_list:
    eventfile_list.append('{0}{1:.0f}/{1:.0f}{2:}/events_{1:.0f}{2:}.csv'.format(events_by_season_dir, thisyear, thisleague.lower()))

# print('Reading these files:')
# pprint(eventfile_list)
# print('\n')

for thisfile in eventfile_list:
    events_i_df = pandas.read_csv(thisfile, encoding='utf-8', index_col='eventID', low_memory=False)#, dtype={'pitch_count': 'str'})
    events_df = pandas.concat((events_df, events_i_df), axis=0)

events_df = events_df.reset_index(drop=True)
events_df.index.name = 'eventID'

events_df = events_df.reset_index()
print('Setting first eventID to be {0:,.0f}...'.format(first_eventID))
events_df.loc[:, 'eventID'] = events_df['eventID'] + first_eventID
events_df = events_df.set_index('eventID')

print('\tfixing pitch counts so they to be properly zero-padded when needed...')
try:
    events_df.loc[:, 'pitch_count'] = events_df['pitch_count'].apply(lambda x: '{0:02d}'.format(x))
except ValueError:
    print("\t\tCAN'T GET PITCH COUNT TO BE AN INTEGER FOR SOME REASON?...")
    events_df.loc[:, 'pitch_count'] = pandas.to_numeric(events_df['pitch_count'], errors='coerce')

print('splitting play into play description and baserunning...')
events_df = events_df.rename(columns={'theplay': 'play_orig'})
events_df = events_df.assign(baserunning = events_df[(events_df['play_orig'].apply(lambda x: '.' in x))]['play_orig'].apply(lambda x: x[x.find('.')+1:]))
events_df = events_df.assign(play_pre_baserunning = events_df['play_orig'].apply(lambda x: x.split('.')[0]))
events_df = events_df.assign(play_description = events_df[(events_df['play_pre_baserunning'].apply(lambda x: '/' in x))]['play_pre_baserunning'].apply(lambda x: x[x.find('/')+1:]))
events_df = events_df.assign(theplay = events_df['play_pre_baserunning'].apply(lambda x: x.split('/')[0]))
events_df = events_df.drop('play_pre_baserunning', axis=1)

print('cleaning baserunning info...')
events_df = events_df.assign(baserunning_cleaned = events_df['baserunning'].dropna().apply(lambda x: x.replace('B','0').replace('H', '4')))


print('creating a new dataframe to hold what we calculate here...')
new_things_df = pandas.DataFrame(data=None, columns=new_columns, index=events_df.index)

# print('backing up...')
# games_df_bk = games_df
# rosters_df_bk = rosters_df
# events_df_bk = events_df
# new_things_df_bk = new_things_df

e = time.time()
g += e-s
print('\n')
print('Read {0:,.0f} games and {1:,.0f} roster entries and {2:,.0f} events in {3:.1f} seconds!'.format(len(games_df), len(rosters_df), len(events_df) ,e-s))
# # crazy game: 
# #games_df.loc['BRO191508312'][['visitor_score', 'home_score']]

# #events_df.sample(1).T

# # # #games_df.loc['BLF191504100'][['visitor', 'visitor_score', 'visitor_linescore', 'home', 'home_score', 'home_linescore']]
# # #games_df[['visitor_linescore', 'home_linescore']][games_df['visitor_linescore'] == '000000001']

#events_df.sample(1).T

Adding game context data for games in 1916...


reading rosters...
reading games...
	fixing linescores...
	checking for DH...
	getting fielder columns...
selecting games and roster rows for 1916...
Setting first eventID to be 104,243...
	fixing pitch counts so they to be properly zero-padded when needed...
		CAN'T GET PITCH COUNT TO BE AN INTEGER FOR SOME REASON?...
splitting play into play description and baserunning...
cleaning baserunning info...
creating a new dataframe to hold what we calculate here...


Read 1,247 games and 549 roster entries and 68,073 events in 15.4 seconds!


In [10]:
def fix_special_case_event(gameID, eventnum, nOuts, onbase, score):
    print('\tFixing a special case in game number {0:,.0f} ({1:}) at event in game {2:.0f}: {3:}'.format(this_game_number, thisrow['gameID'], thisrow['event_in_game'], thisrow['play_orig']))
    if ((gameID == 'PHA191509242') and (eventnum == 17)):    
        print('\t\tExplanation: weavb101 stole at 2nd then came around to score on an error\n')
        score[0] += 1
        onbase = {1: '', 2: '', 3: ''}
    if ((gameID == 'CHN191509242') and (eventnum == 37)):
        print('\t\tExplanation: mccaa103 caught stealing at 2nd then willc103 returned to 2nd after deciding not to steal 3rd\n')
        nOuts += 1
        onbase = {1: '', 2: 'willc103', 3: ''}
    if ((gameID == 'PHI191509092') and (eventnum == 40)):    
        print('\t\tExplanation: fielding error but paskd101 still out at 3rd\n')
        onbase[3] = ''
        nOuts += 1
    if ((gameID == 'PIT191506080') and (eventnum == 47)):
        print('\t\tExplanation: hincb101 steals home, wagnh101 stals 2nd, batter vioxj101 tries to steal home but gets caught')
        print('\t\t\tExplanation: Thus, code failed to account for hincb101 steal of home from 3rd\n')
        score[1] += 1
    if ((gameID == 'PHA191605170') and (eventnum == 24)):    
        print('\t\tExplanation: walsj108 steals home and scores; pickc103 steals third but it caught at home for an out\n')
        score[vh] += 1
        
# #     if ((gameID == 'PIT191606242') & (eventnum == 58)):
# #         print('\t\tExplanation: steal of home was counted in both baserunning and stolen base event\n')
# #         score[1] = 4
# #     if ((gameID == 'SLN191606290') & (eventnum == 5)):
# #         print('\t\tExplanation: steal of home by zeidr101 was double-counted\n')
# #         score[0] = 1
# #     if ((gameID == 'BOS191707261') & (eventnum == 49)):
# #         print('\t\tExplanation: hooph101 and scote101 both scored on failed tag at home\n')
# #         score[1] = 6
# #         onbase[3] = ''
# #     if ((gameID == 'CHA191907100') & (eventnum == 52)): 
# #         score[1] = 6
# #         print('\t\tExplanation: mcmuf101 scored due to PHA C error at home')
# #     if ((gameID == 'PHI191710022') and (eventnum == 25)):
# #         print('\t\tExplanation: zimmh101 still out at home despite error\n')
# #         nOuts += 1
# #         onbase[1] = ''
# #         score[0] = 1
# #     if ((gameID == 'CHA191806200') & (eventnum == 67)):
# #         print('\t\tExplanation: felsh101 out on fielders choice after run scored\n')
# #         nOuts += 1
# #     if ((gameID == 'DET191906021') & (eventnum == 28)):
# #         print('\t\tExplanation: felsh101 stole home on error by CHA 2B\n')
# #         score[0] += 1
# #     if ((gameID == 'NY1191908261') & (eventnum == 31)):
# #         print('\t\tExplanation: kaufb101 steals second only, and does not score\n')
# #         onbase = {1: '', 2: 'kaufb101', 3: ''}
# #         score[1] += -1
# #     if ((gameID == 'NY1192008151') & (eventnum == 66)):
# #         print('\t\tExplanation: boect101 scored on error at home\n')
# #         onbase = {1: 'rudod101', 2: 'fordh101', 3: ''}
# #     if ((gameID == 'PHI192005060') & (eventnum == 5)):
# #         print('\t\tExplanation: cruiw101 scored on error at home by PHI C\n')
# #         score[0] += 1
# #     if ((gameID == 'PHI192009100') & (eventnum == 53)):
# #         print('\t\tExplanation: mckeb101 steal of home was double-counted\n')
# #         score[0] += -1
# # #     if ((gameID == 'CHN192108032') & (eventnum == 63)):
# # #         print('\t\tExplanation: power101 steal of home was double-counted, also barbw101 stole 2nd \n')
# # #         score[0] += -1
# # #         onbase = {1: '', 2: 'barbw101', 3: ''}
# # #     if ((gameID == 'CIN192104180') & (eventnum == 31)):
# # #         print('\t\tExplanation: fonsl101 steal of home was double-counted, also crans102 stole 2nd \n')
# # #         score[1] += -1
# # #         onbase = {1: '', 2: 'crans102', 3: ''}
# # #     if ((gameID == 'CIN192104240') & (eventnum == 55)):
# # #         print('\t\tExplanation: carem101 steal of home was double-counted, also marar101 stole 2nd \n')
# # #         score[0] += -1
# # #         onbase = {1: '', 2: 'marar101', 3: ''}
# # #     if ((gameID == 'CIN192106110') & (eventnum == 26)):
# # #         print('\t\tExplanation: rouse101 steal of home was double-counted, also duncp101 stole 2nd \n')
# # #         score[1] += -1        
# # #         onbase = {1: '', 2: 'duncp101', 3: ''}
# # #     if ((gameID == 'NY1192105040') & (eventnum == 39)):        
# # #         print('\t\tExplanation: ruetd101 steal of home was double-counted, also olsoi101 stole 2nd \n')
# # #         score[0] += -1        
# # #         onbase = {1: '', 2: 'olsoi101', 3: ''}
# # #     if ((gameID == 'NY1192105240') & (eventnum == 22)):        
# # #         print('\t\tExplanation: carem101 steal of home was double-counted, also bigbc101 stole 2nd \n')
# # #         score[0] += -1        
# # #         onbase = {1: '', 2: 'bigbc101', 3: ''}
# # #     if ((gameID == 'NY1192105302') & (eventnum == 85)):        
# # #         print('\t\tExplanation: carem101 steal of home was double-counted\n')
# # #         score[1] += -1        
# # #     if ((gameID == 'NY1192106210') & (eventnum == 57)):        
# # #         print('\t\tExplanation: power101 steal of home was double-counted, also barbw101 stole 2nd\n')
# # #         score[0] += -1        
# # #         onbase = {1: '', 2: 'barbw101', 3: ''}
                    
# # #     if ((gameID == 'ARI202107310') & (eventnum == 16)):
# # #         print('\t\tExplanation: steal of home by taylc001 was double-counted\n')
# # #         score[0] = 3
# # #     if ((gameID == 'ARI202108300') & (eventnum == 5)):        
# # #         print('\t\tExplanation: tatif002 scoring on his own HR was double-counted\n')
# # #         score[0] = 2
# # #     if ((gameID == 'ATL202106201') & (eventnum == 33)):
# # #         print('\t\tExplanation: freef001 steal of home was double-counted\n')
# # #         score[1] = 1
# # #     if ((gameID == 'CHN202107080') and (eventnum == 13)):
# # #         print('\t\tExplanation: realj001 steal of home was double-counted\n')
# # #         score[0] = 1
# # #         return nOuts, onbase, score
# # #     if ((gameID == 'CIN202104160') and (eventnum == 32)):
# # #         print('\t\tExplanation: vottj001 scoring on his own HR was double-counted\n')
# #         score[1] = 7
    return nOuts, onbase, score
    
print('ok')

ok


In [12]:
start_at_game_number = 244
pai = 5
pfe = -1

s = time.time()
# print('getting from backup...')
# games_df = games_df[games_df['gameno'].apply(lambda x: True == False)]
# rosters_df = rosters_df[rosters_df['theyear'].apply(lambda x: True == False)]
# events_df = events_df[events_df['event_in_game'].apply(lambda x: True == False)]
# games_df = games_df_bk
# rosters_df = rosters_df_bk
# events_df = events_df_bk

base_updater_atom_re = re.compile('(\d)+\((\d|B|H)\)')

fielder_out_re = re.compile('^\d+!*')

hit_re = re.compile('^(S|D|T|HR)(\d|GR)*$')
onbase_nohit_re = re.compile('^(\d|FL)*(W|IW|HP|C|E|FC)')

stolen_base_re = re.compile('SB(\d|H)')
caught_stealing_re = re.compile('CS(\d|H)')
pickoff_re = re.compile('PO\d\(*E*\d*\)')

error_re = re.compile('^(\d|FL)*E\d+')
#baserunning_fielder_error_re = re.compile('(\d|B|H)\(\d*(E|PB)\d*\)')  # had to add the possibility of (NR)(*E)
baserunning_fielder_error_re = re.compile('(?<!\))\(\d*E\d*\)')


nr_ur_fixer_re = '(\((NR|UR|NORBI)\))+'

print_these = {}
print_these['gamelog'] = True
print_these['events'] = True
print_these['baserunning'] = True
print_these['noplay'] = False

# for x in print_these.keys():
#     if (print_these[x]):
#         print('\tprinting {0:} events...'.format(x))
#     else:
#         print('\tNOT printing {0:} events...'.format(x))

# print('printing starting with the {0:} inning...'.format(ordinaltg(pai)))
# print('\n')

killswitch = False
max_event_id = np.max(new_things_df.index.tolist())

courtesy_runner_half_inning = -1
score = [0,0]
onbase = {1: '', 2: '', 3: ''}

# NOTE: we HAVE TO keep all the NPs because pitches occurred on all of them!
print('\nparsing plays...')

# #gamelist = sorted(events_df['gameID'].unique().tolist())
gamelist = events_df['gameID'].unique().tolist()
print('Found {0:,.0f} events from {1:,.0f} games-with-events!'.format(len(events_df), len(gamelist)))
print('\n')

# nGames = len(gamelist)

for this_game_number in range(start_at_game_number, len(gamelist)):
    thisgameID = gamelist[this_game_number]
    already_said_ended_early = False
    if not(print_these['gamelog']):
        if (np.mod(this_game_number, 100) == 0):
            print('parsing game {0:.0f} ({1:}): {2:} at {3:} on {4:}...'.format(this_game_number, thisgameID, games_df.loc[thisgameID]['visitor'], games_df.loc[thisgameID]['home'], games_df.loc[thisgameID]['date'].strftime('%Y-%m-%d')))
    else:
        if (np.mod(this_game_number, 1) == 0):
            print('parsing game {0:.0f} ({1:}): {2:} at {3:} on {4:}...'.format(this_game_number, thisgameID, games_df.loc[thisgameID]['visitor'], games_df.loc[thisgameID]['home'], games_df.loc[thisgameID]['date'].strftime('%Y-%m-%d')))

    for i in [0,1]:
        score[i] = 0    
    this_game_innings = events_df[events_df['gameID'] == thisgameID]['inning'].unique()              
    for this_inning in this_game_innings:
        for vh in [0,1]:
            nOuts = 0
            for i in onbase:
                onbase[i] = ''    
            if (vh == 0):
                inningside = 'top'
            else:
                inningside = 'bottom'
            if ((print_these['gamelog']) & (this_inning >= pai)):
                print('{0:} of the {1:}...'.format(inningside.title(), ordinaltg(this_inning)))
            for ix, thisrow in events_df[(events_df['gameID'] == thisgameID) & (events_df['inning'] == this_inning) & (events_df['visitor_or_home'] == vh)].iterrows():
                new_things_df.loc[ix, ['vis_score', 'home_score', 'outs', 'runner_1b', 'runner_2b','runner_3b'
                                      ]] = [score[0], score[1], nOuts, onbase[1], onbase[2], onbase[3]]
                batter_event = True
                weird = False
                manual_batter_advance = False
                explicit_baserunning = False
                explicit_baserunning_events = []  #  will contain dictionaries like this: {'start_base': np.nan, 'end_base': np.nan}
                is_forceout = False
                
                if (type(thisrow['play_description']) == str):
                    if ('COUR' in thisrow['play_description']):
                        courtesy_runner_half_inning = vh

                # No play?
                if (thisrow['theplay'] == 'NP'):
                    if ((print_these['noplay']) & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                        print('-------------- NO PLAY --------------')
                    explained = False
                    if (type(thisrow['comments']) == str):
                        if (print_these['noplay'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                            print('\tEvent {0:0d} (ix={1:0d}): comment: {2:}'.format(thisrow['event_in_game'], ix, thisrow['comments']))
                            print('\n')
                        explained = True
                    if (type(thisrow['pinch_runner_playerID']) == str):
                        if (print_these['noplay'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                            print('\tEvent {0:0d} (ix={1:0d}): pinch runner: {2:} replaces {3:}'.format(thisrow['event_in_game'], ix, thisrow['pinch_runner_playerID'], thisrow['pinch_runner_replaced_playerID']))
                            print('\n')
                        explained = True
                    if (thisrow['batterID'] != events_df.loc[ix+1]['batterID']):
                        if (print_these['noplay'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                            print('\tEvent {0:0d} (ix={1:0d}): {2:} replaces {3:} at bat...'.format(thisrow['event_in_game'], ix, events_df.loc[ix+1]['batterID'], thisrow['batterID']))
                            print('\n')
                        explained = True
                    if (thisrow['pitcherID'] != events_df.loc[ix+1]['pitcherID']):
                        if (print_these['noplay'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                            print('\tEvent {0:0d} (ix={1:0d}): {2:} replaces {3:} at pitcher...'.format(thisrow['event_in_game'], ix, events_df.loc[ix+1]['pitcherID'], thisrow['pitcherID']))
                            print('\n')
                        explained = True    
                    for this_fielder in fielder_columns:
                        if (thisrow[this_fielder] != events_df.loc[ix+1][this_fielder]):
                            if (print_these['noplay'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                                print('\tEvent {0:0d} (ix={1:0d}): {2:} replaces {3:} at pitcher...'.format(thisrow['event_in_game'], ix, events_df.loc[ix+1]['pitcherID'], thisrow['pitcherID']))
                                print('\n')
                            explained = True
                    if (courtesy_runner_half_inning != vh):
                        explained = True
                        if (vh != courtesy_runner_half_inning):
                            courtesy_runner_half_inning = -1
                    if (not(explained)):
                        print('\tEvent {0:0d} (ix={1:0d}): UNEXPLAINED NO PLAY'.format(thisrow['event_in_game'], ix))
                        print('This row:')
                        print(thisrow)
                        print('\n')
                    if (print_these['noplay'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                        print('-------------- /NO PLAY --------------')
                else:    # there is a play
                    if ((print_these['events']) & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                        print('\tEvent {0:0d} (ix = {1:.0f}): {2:} at bat with {3:.0f} outs with score [V {4:0d} H {5:0d}] and onbase = {6:}...'.format(thisrow['event_in_game'], ix, thisrow['batterID'], nOuts, score[0], score[1], onbase))
                        print('\t\tPLAY: {0:} ({1:})...'.format(thisrow['theplay'], thisrow['play_orig']))
                    if ((re.search(base_updater_atom_re, thisrow['theplay']) != None) and ('CS' not in thisrow['theplay']) and ('PO' not in thisrow['theplay'])):
                        if (print_these['events'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                            print('\t\t\tPlay found by base updater atom method in play: {0:} ({1:})...'.format(thisrow['theplay'], thisrow['play_orig']))
                        for this_base_updater_atom in re.finditer(base_updater_atom_re, thisrow['theplay']):
                            if (print_these['events'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                                print('\t\t\t\t{0:}'.format(
                                    #thisrow['theplay'][0:this_base_updater_atom.start()], 
                                    thisrow['theplay'][this_base_updater_atom.start():this_base_updater_atom.end()]#,
                                    #thisrow['theplay'][this_base_updater_atom.end():]
                                ))
                            runner_from_out_str = thisrow['theplay'][this_base_updater_atom.start():this_base_updater_atom.end()][re.search("\(",thisrow['theplay'][this_base_updater_atom.start():this_base_updater_atom.end()]).start()+1:re.search("\)",thisrow['theplay'][this_base_updater_atom.start():this_base_updater_atom.end()]).end()-1]
                            if (runner_from_out_str == 'B'):
                                if (print_these['events'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                                    print('\t\t\t\tbatter out!')
                                manual_batter_advance = True
                                nOuts += 1
                            else:
                                runner_from_base_out = int(runner_from_out_str)
                                if (print_these['events'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                                    print('\t\t\t\trunner coming from {0:} ({1:}) is out!'.format(ordinaltg(runner_from_base_out), onbase[runner_from_base_out]))
                                nOuts += 1
                                onbase[runner_from_base_out] = ''
                                if (not(manual_batter_advance)):
                                    if ('FO' in thisrow['play_description']):
                                        if (print_these['events'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                                            print('\t\t\t\t\tThis is a forceout!')
                                        is_forceout = True
                        if ((print_these['events']) & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                            print(re.findall(base_updater_atom_re, thisrow['theplay']))
                        #if ((len(re.findall(base_updater_atom_re, thisrow['theplay'])) > 1) & ('DP' in thisrow['play_description']) & ~(manual_batter_advance)):
                        if ((len(re.findall(base_updater_atom_re, thisrow['theplay'])) > 1) & ~(manual_batter_advance)):
                            is_forceout = True
                        if (print_these['events'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                            print('{0:.0f} events long...'.format(len(re.findall(base_updater_atom_re, thisrow['theplay']))))
                            print('\n')
                    

                    #print('update baserunning for batter events...')
                    if (type(thisrow['baserunning_cleaned']) == str):
                        if (thisrow['theplay'][0:2] == 'HR'):
                            explicit_baserunning = True
                        if ((print_these['baserunning']) & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                            print('--------------------')
                            print('parsing baserunning: {0:} <{1:}>...'.format(thisrow['baserunning_cleaned'], thisrow['play_orig']))
                            print('STATE BEFORE: V {0:.0f} H {1:.0f}, {2:.0f} outs, onbase: {3:}'.format(score[0], score[1], nOuts, onbase))
                        
                        
                        if (re.search(nr_ur_fixer_re, thisrow['baserunning_cleaned']) != None):
                            baserunning_string = thisrow['baserunning_cleaned'][0:re.search(nr_ur_fixer_re, thisrow['baserunning_cleaned']).start()]+thisrow['baserunning_cleaned'][re.search(nr_ur_fixer_re, thisrow['baserunning_cleaned']).end():]
                        else:
                            baserunning_string = thisrow['baserunning_cleaned']
                        # parse baserunning string and load information into a dataframe
                        raw_advances = baserunning_string.split(';')
                        
                        parsing_advances = {}
                        advances = []
                        for p in range(0, len(raw_advances)):
                            parsing_advances[raw_advances[p][0]] = {'start_base': int(raw_advances[p][0]), 'successmark': raw_advances[p][1], 'end_base': int(raw_advances[p][2])}
                            if (len(raw_advances[p]) > 3):
                                parsing_advances[raw_advances[p][0]]['more_info'] = raw_advances[p][3:]
                            else:
                                parsing_advances[raw_advances[p][0]]['more_info'] = np.nan
                            
                        keyorder = sorted(list(parsing_advances), reverse=True)
                        for q in keyorder:
                            this_parsing_advance = {}
                            this_parsing_advance['start_base'] = parsing_advances[q]['start_base']
                            this_parsing_advance['end_base'] = parsing_advances[q]['end_base']
                            if (parsing_advances[q]['successmark'] == '-'):
                                this_parsing_advance['successful'] = True
                            elif (parsing_advances[q]['successmark'] == 'X'):
                                this_parsing_advance['successful'] = False
                            else:
                                print('DANGER: success mark is weird for {0:}'.format(thisrow['baserunning_cleaned']))
                                killswitch = True
                            this_parsing_advance['more_info'] = parsing_advances[q]['more_info']
                            advances.append(this_parsing_advance)
                        
                        for this_advance in advances:
                            if (type(this_advance['more_info']) == str):
                                if ((print_these['baserunning']) & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                                    print('========= MORE INFO: {0:} ==========='.format(this_advance['more_info']))
                            if (this_advance['successful']):
                                if (this_advance['end_base'] == 4):
                                    score[vh] += 1
                                    explicit_baserunning_events.append({'start_base': this_advance['start_base'], 'end_base': this_advance['end_base']})
                                else:
                                    if (this_advance['start_base'] == 0):
                                        onbase[this_advance['end_base']] = thisrow['batterID']
                                    else:
                                        onbase[this_advance['end_base']] = onbase[this_advance['start_base']]
                            elif (~this_advance['successful']):
                                if (re.search(baserunning_fielder_error_re, this_advance['more_info'])):
                                    explicit_baserunning = True
                                    if ((print_these['baserunning']) & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                                        print('FIELDER ERROR! BASERUN SUCCEEDS! {0:}'.format(this_advance['more_info']))
                                    if (this_advance['end_base'] == 4):
                                        score[vh] += 1
                                        explicit_baserunning_events.append({'start_base': this_advance['start_base'], 'end_base': this_advance['end_base']})
                                    else:
                                        if (this_advance['start_base'] == 0):
                                            onbase[this_advance['end_base']] = thisrow['batterID']
                                        else:
                                            onbase[this_advance['end_base']] = onbase[this_advance['start_base']]
                                else:
                                    nOuts += 1
                            if (this_advance['start_base'] == 0):
                                manual_batter_advance = True
                            else:
                                onbase[this_advance['start_base']] = ''
                            explicit_baserunning_events.append({'start_base': this_advance['start_base'], 'end_base': this_advance['end_base']})
                        if ((print_these['baserunning']) & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                            print('STATE AFTER: V {0:.0f} H {1:.0f}, {2:.0f} outs, onbase: {3:}'.format(score[0], score[1], nOuts, onbase))
                            print('--------------------')
                        
                    #if (('SB' in thisrow['theplay']) | ('DI' in thisrow['theplay'])):


                    if ('SB' in thisrow['theplay']):
                        if (print_these['events'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                            print('stolen base(s): {0:}'.format(thisrow['theplay']))
                            print('onbase BEFORE: {0:}'.format(onbase))
                            
                        ## stolen bases should be listed highest base first, so swap if needed
                        if (thisrow['theplay'] == 'SB2;SBH'):
                            ordered_play = ['SBH', 'SB2']
                        else:
                            ordered_play = thisrow['theplay'].split(';')
                        for this_steal in ordered_play:
                            base_stolen_str = this_steal[re.search(stolen_base_re, this_steal).end()-1:re.search(stolen_base_re, this_steal).end()]
                            if (base_stolen_str == 'H'):
                                base_stolen = 4
                            else:
                                base_stolen = int(base_stolen_str)
                            if (print_these['events'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                                print('{0:} base is stolen'.format(ordinaltg(base_stolen)))
                            if (base_stolen not in [x['end_base'] for x in explicit_baserunning_events]):
                                if (base_stolen == 4):
                                    score[vh] += 1
                                else:                            
                                    for b in range(base_stolen-1, 0, -1):
                                        if (onbase[b] != ''):
                                            onbase[base_stolen] = onbase[b]
                                            break
                                for b in range(base_stolen-1, 0, -1):
                                    if (onbase[b] != ''):
                                        onbase[b] = ''
                                        break

                        if (print_these['events'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                            print('stolen base(s): {0:}'.format(thisrow['theplay']))
                            print('onbase AFTER: {0:}'.format(onbase))

                            
                    if ('CS' in thisrow['theplay']):
                        if (print_these['events'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                            print('onbase before: {0:}'.format(onbase))
                            print('caught stealing!')
                            print(thisrow['theplay'])
                        
                        for this_caught in thisrow['theplay'].split(';'):
                            this_caught = re.sub(nr_ur_fixer_re, '', this_caught)
                            caught_at_str = this_caught[re.search(caught_stealing_re, this_caught).end()-1:re.search(caught_stealing_re, this_caught).end()]
                            if (caught_at_str == 'H'):
                                caught_at = 4
                            else:
                                caught_at = int(caught_at_str)
                            if (caught_at not in [x['end_base'] for x in explicit_baserunning_events]):
                                if (re.search(baserunning_fielder_error_re, this_caught) == None):  # successful cuaght stealing
                                    if (print_these['events'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                                        print('\tRunner caught at {0:}'.format(ordinaltg(caught_at)))
                                    nOuts += 1 
                                    onbase[caught_at] = ''
                                    for b in range(caught_at-1,0,-1):
                                        if (onbase[b] != ''):
                                            onbase[b] = ''
                                else: # failed cuaght stealing
                                    if (print_these['events'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                                        print('\tFailed caught stealing at {0:}'.format(ordinaltg(caught_at)))
                                    if (caught_at == 4):
                                        score[vh] += 1
                                    else:
                                        for b in range(caught_at-1, 0,-1):
                                            onbase[caught_at] = onbase[b]
                                            onbase[b] = ''
                                        break

                        if (print_these['events'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                            print('onbase after: {0:}'.format(onbase))
                            
                    if ('PO' in thisrow['theplay']):
                        if (print_these['events'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                            print('pickoff!')
                            print(thisrow['theplay'])
                        for match in re.finditer(pickoff_re, thisrow['theplay']):
                            if (re.search(baserunning_fielder_error_re, thisrow['theplay']) != None):
                                if (print_these['events'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                                    print('Failed pickoff: {0:}'.format(thisrow['theplay']))
                            else:
                                picked_off_at = int(thisrow['theplay'][match.start()+2:match.start()+3])
                                if (print_these['events'] & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                                    print('picked off at {0:}... <{1:}>'.format(ordinaltg(picked_off_at), thisrow['theplay'][match.start():match.end()]))
                                onbase[picked_off_at] = ''
                                nOuts += 1
                    if (thisrow['theplay'][0] == 'K'):
                        if not(manual_batter_advance):
                            nOuts += 1
                    elif (((thisrow['theplay'][0] == 'W') & (thisrow['theplay'][0:2] != 'WP')) | (thisrow['theplay'][0:2] == 'HP') | (thisrow['theplay'][0:2] == 'IW') | ((thisrow['theplay'][0] == 'C') & (thisrow['theplay'][0:2] != 'CS'))):
                        if not(manual_batter_advance):
                            onbase[1] = thisrow['batterID']
                    elif ((thisrow['theplay'][0] == 'S') & (thisrow['theplay'][0:2] != 'SB')):
                        if not(manual_batter_advance):
                            onbase[1] = thisrow['batterID']
                    elif ((thisrow['theplay'][0] == 'D') & (thisrow['theplay'][0:2] != 'DI')):
                        if not(manual_batter_advance):
                            onbase[2] = thisrow['batterID']
                    elif (thisrow['theplay'][0] == 'T'):
                        if not(manual_batter_advance):
                            onbase[3] = thisrow['batterID']
                    elif (thisrow['theplay'][0:2] == 'HR'):
#                        print(onbase)
                        score[vh] += len([x for x in onbase.keys() if onbase[x] != ''])
#                         print(manual_batter_advance)
#                         print(score)
                        if (manual_batter_advance == False):
                            score[vh] += 1
#                        print(score)
                        for i in [1,2,3]:
                            onbase[i] = ''
                            
                    elif (re.search(error_re, thisrow['theplay']) != None):
                        if not(manual_batter_advance):
                            if (not(thisrow['theplay'][0:3] == 'FLE')):   # If the error was on a foul ball, the runner is not out
                                onbase[1] = thisrow['batterID']
                            if ((print_these['baserunning']) & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                                print('ERROR ON PLAY, batter advances!')
                        
                    elif (re.search(fielder_out_re, thisrow['theplay'])):
                        if not(manual_batter_advance):
                            if (is_forceout):
                                onbase[1] = thisrow['batterID']
                            elif (nOuts < 3):
                                nOuts += 1
                    elif (thisrow['theplay'][0:2] == 'FC'):
                        if not(manual_batter_advance):
                            onbase[1] = thisrow['batterID']
                    elif (thisrow['theplay'][0:2] in ['SB', 'CS', 'PO', 'BK', 'OA', 'WP', 'PB', 'DI']):
                        pass
                    else:
                        print('PLAY NOT FOUND!')
                        print('{0:} ({1:})'.format(thisrow['theplay'], thisrow['play_orig']))
                        killswitch = True
                    if (thisrow['gameID'] in list(special_case_events.keys())):
                        for this_special_event_num in special_case_events[thisrow['gameID']]:
                            if (thisrow['event_in_game'] == this_special_event_num):
                                nOuts, onbase, score = fix_special_case_event(thisrow['gameID'], thisrow['event_in_game'], nOuts, onbase, score)
                    if ((print_these['events']) & (this_inning >= pai) & (thisrow['event_in_game'] >= pfe)):
                        print('\t\t\tOUTCOME: {0:} at bat with {1:.0f} outs with score [V {2:0d} H {3:0d}] and onbase = {4:}...'.format(thisrow['batterID'], nOuts, score[0], score[1], onbase))
                        print('\n')
                                
            # checks: each half-inning must end with 3 outs, and each inning must end with scores matching game log
            if (nOuts != 3):
                if (thisrow['gameID'] in early_end_game_list):
                    if (not(already_said_ended_early)):
                        print('\tGAME NUMBER {0:.0f} ENDED EARLY: {1:}'.format(this_game_number, thisrow['gameID']))
                        print('\n')
                        already_said_ended_early = True
                else:
                    if not((this_inning == np.max(this_game_innings)) & (vh == 1)):
                        print('DISCREPANCY: {0:} of the {1:} has {2:0d} outs!'.format(inningside.title(), ordinaltg(this_inning), nOuts))
                        killswitch = True
                        max_event_id = ix
            skips = 0
            sco = 0
            if (games_df.loc[thisgameID]['has_linescores']):
                if (vh == 0):
                    for j in range(0, this_inning):
                        if (games_df.loc[thisgameID]['visitor_linescore'][j] == '('):
                            visitor_linescore_raw = games_df.loc[thisgameID]['visitor_linescore']
                            skips = visitor_linescore_raw[j:].find(')')
                            sco += int(visitor_linescore_raw[visitor_linescore_raw.find('(')+1:visitor_linescore_raw.find(')')])
                        else:
                            sco += int(games_df.loc[thisgameID]['visitor_linescore'][j+skips])
                    if (score[0] != sco):
                        print('SCORE DISCREPANCY: {0:} of the {1:}: {2:} has {3:0d} and they should have {4:0d} <linescore: {5:}>'.format(inningside, ordinaltg(this_inning), games_df.loc[thisgameID]['visitor'], score[0], sco, games_df.loc[thisgameID]['visitor_linescore']))
                        killswitch = True
                        max_event_id = ix
                else:
                    for j in range(0, this_inning):
                        if (games_df.loc[thisgameID]['home_linescore'][j+skips] != 'x'):
                            if (games_df.loc[thisgameID]['home_linescore'][j] == '('):
                                home_linescore_raw = games_df.loc[thisgameID]['home_linescore']
                                skips = home_linescore_raw[j:].find(')')
                                sco += int(home_linescore_raw[home_linescore_raw.find('(')+1:home_linescore_raw.find(')')])
                            else:
                                sco += int(games_df.loc[thisgameID]['home_linescore'][j+skips])
                    if (score[1] != sco):
                        print('SCORE DISCREPANCY: {0:} of the {1:}: {2:} has {3:0d} and they should have {4:0d} <linescore: {5:}>'.format(inningside, ordinaltg(this_inning), games_df.loc[thisgameID]['home'], score[1], sco, games_df.loc[thisgameID]['home_linescore']))
                        killswitch = True
                        max_event_id = ix
                #killswitch = False
    #             if ((print_these['gamelog']) & (this_inning >= pai)):
    #                 print('\n')
            
            if (killswitch):
                break
        if (killswitch):
            break
    if (killswitch):
        break

events_contexts_df = pandas.concat((events_df.loc[events_df.index <= max_event_id], new_things_df.loc[new_things_df.index <= max_event_id]), axis=1)
e = time.time()
g += e-s
print('\n')
print('Processed {0:,.0f} events in {1:.0f} minutes {2:.0f} seconds!'.format(len(events_contexts_df), np.floor((e-s)/60), (e-s)%60))



parsing plays...
Found 68,073 events from 819 games-with-events!


parsing game 244 (PHA191606280): NYA at PHA on 1916-06-28...
Top of the 5th...
	Event 38 (ix = 124598): keatr101 at bat with 0 outs with score [V 3 H 2] and onbase = {1: '', 2: '', 3: ''}...
		PLAY: K (K)...
			OUTCOME: keatr101 at bat with 1 outs with score [V 3 H 2] and onbase = {1: '', 2: '', 3: ''}...


	Event 39 (ix = 124599): gilhf101 at bat with 1 outs with score [V 3 H 2] and onbase = {1: '', 2: '', 3: ''}...
		PLAY: W (W)...
			OUTCOME: gilhf101 at bat with 1 outs with score [V 3 H 2] and onbase = {1: 'gilhf101', 2: '', 3: ''}...


	Event 40 (ix = 124600): highh101 at bat with 1 outs with score [V 3 H 2] and onbase = {1: 'gilhf101', 2: '', 3: ''}...
		PLAY: SB2 (SB2)...
stolen base(s): SB2
onbase BEFORE: {1: 'gilhf101', 2: '', 3: ''}
2nd base is stolen
stolen base(s): SB2
onbase AFTER: {1: '', 2: 'gilhf101', 3: ''}
			OUTCOME: highh101 at bat with 1 outs with score [V 3 H 2] and onbase = {1: '', 2: 'gilhf101',

In [ ]:
s = time.time()
events_contexts_df = pandas.concat((events_df, new_things_df), axis=1)
#new_things_df
events_contexts_df.columns.tolist()

events_contexts_df = events_contexts_df[final_column_order]
events_contexts_df.to_csv('{0:0d}.csv'.format(thisyear))
e = time.time()
g += e-s
print('Processed {0:,.0f} events in {1:.0f} minutes {2:.0f} seconds!'.format(len(new_things_df), np.floor(g/60), g%60))


In [ ]:
#games_df.loc['BLF191504100'][['visitor_linescore', 'home_linescore', 'has_linescores']]#.groupby('has_linescores').size()